## Notebook / 4 May 2021 (Example Data)

a-2021-05-04.ipynb

## Input Data

In [1]:
import pandas as pd
import paper as pa
import standard_data as sdata
TRESHOLD = 92
df = sdata.input_set
#df = sdata.input_set = pd.read_csv("data/t25_completolda_documenti.csv")
#df = df.drop(columns=['volume','issue','autori','data','year'])
df.head(3).style.set_caption("<b style='background-color: yellow;'>Table 0: Input Data</b>")

,paper_key,t1,t2,t3,t4
0,p1,10,20,61,9
1,p2,80,2,8,10
2,p3,30,60,4,6


## Example Extraction for p2

In [2]:
pa.topics_that_are_sufficient_to_reach_the_threshold(df,rownum = 1, treshold=TRESHOLD)['as_df']


,topic,weight,cumsum
0,t1,80,80
1,t4,10,90
2,t3,8,98


`topics_that_are_sufficient_to_reach_the_threshold` fonksiyonu ne yapıyor: Önce girdi tablosundaki değerleri (p2 	80 	2 	8 	10) alıyor sonra sayılara göre sıralıyor t konuları da ona göre sıralanıyor.  Sonra kümülatif değerleri alıyor

## Table 1: Analysis for each paper

In [3]:
# columns
notta = 'number of topics that are sufficient to reach the threshold'
lotta = 'label of topics that are sufficient to reach the threshold'

for index,row in df.iterrows():
    reat = pa.topics_that_are_sufficient_to_reach_the_threshold(df,rownum = index,treshold=TRESHOLD)
    df.at[index,notta] = reat['as_count']
    df.at[index,lotta] = reat['as_topic_labels']
    #max min (TODO:Look table 4, maxweight is 40 and min weight 25)
    df.at[index,"max weight"] = max(reat['as_df'].weight)
    df.at[index,"min weight"] = min(reat['as_df'].weight)

In [4]:
# styling
df['max weight'] = df['max weight'].astype(int)
df['min weight'] = df['min weight'].astype(int)
df[notta] = df[notta].astype(int)
table_1= df
table_1.head(3)

,paper_key,t1,t2,t3,t4,number of topics that are sufficient to reach the threshold,label of topics that are sufficient to reach the threshold,max weight,min weight
0,p1,10,20,61,9,4,"t3,t2,t1,t4",61,9
1,p2,80,2,8,10,3,"t1,t4,t3",80,8
2,p3,30,60,4,6,3,"t2,t1,t4",60,6


Ne yaptık: `topics_that_are_sufficient_to_reach_the_threshold` fonksiyonu alınıp buraddaki t listesi `label of topics that are sufficient to reach the threshold` kolonuna yazdırılıyor. Sonra bu kolonda geçen konu sayısı `number of topics that are sufficient to reach the threshold` kolonuna yazdırılıyor. Burada mesela 4 demek o makalede 4 konu geçmiş demek, 3 demek 3 konu geçmiş demek. Max ve min weight kolonları ise normal weight'lerden alınıyor

## Table 2: Summary of the number of topics (Old Reat Table)

In [5]:
t_reat_freq = table_1[notta].value_counts().to_frame().fillna(0).reset_index()
t_reat_perc = round(table_1[notta].value_counts(normalize = True)*100,2).to_frame().reset_index()
t_reat_dist = t_reat_freq.merge(t_reat_perc,on="index")
t_reat_dist.columns = ['topic','reat_freq','reat_perc']
t_reat_dist['cms'] = t_reat_dist.reat_perc.cumsum()
t_reat_dist.columns = ['number of topic','number of paper','percentage','cumulative percentage']
t_reat_dist

,number of topic,number of paper,percentage,cumulative percentage
0,3,3,75.0,75.0
1,4,1,25.0,100.0


Ne yaptık: Burada ise Table 1'deki `number of topics that are sufficient to reach the threshold` kolonunun frekans değerleri alındı. Yani içinde 3 konu geçen 3 makale, 4 konu geçen 1 makale var. Kalan kolonlar da buna göre hesaplandı

## Table 3: Summary of the topic distribution (Old  Suffi Table)

In [6]:
def get_order(topicno):
    return topicno.split("t")[1]

In [7]:
melted_topics = pd.DataFrame(",".join(table_1[lotta].to_list()).split(","))
tfreq = melted_topics[0].value_counts().to_frame().reset_index()
tfreq.columns = ['topic','number of paper']
count_of_papers_in_corpus = table_1.count()[0]
number_of_papers_in_corpus = tfreq['number of paper']/count_of_papers_in_corpus
tfreq['number of paper in corpus'] = number_of_papers_in_corpus
tfreq['order'] = tfreq['topic'].apply(get_order).astype(int)
tfreq.sort_values(by = "order")

,topic,number of paper,number of paper in corpus,order
1,t1,4,1.00,1
3,t2,2,0.50,2
2,t3,3,0.75,3
0,t4,4,1.00,4


Ne yaptı: Burada Tablo 1'de geçen makale sayısı `count_of_papers_in_corpus` ve her bir konunun table1'de kaç kez geçtiğini kullanarak bu tabloyu oluşturduk.